#Installing Gardio Library

In [ ]:
pip install gradio openai pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


##Importing Libraries

In [ ]:
import os
import time
import gradio as gr
from openai import OpenAI
import pandas as pd
from datetime import datetime

##Set Environmental Variables

In [ ]:
# Set up your API key
os.environ["IDA_LLM_API_KEY"] = "your key here"

# Initialize your private LLM client
client = OpenAI(
    base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1",
    api_key=os.environ['IDA_LLM_API_KEY']
)

##Database Initialization

In [ ]:
def initialize_excel_database():
    '''
    Initialize Excel database file for storing chat interaction logs and metrics.
    Creates new Excel file with structured columns for timestamps, queries, responses, and performance data if not exists.
    '''
    filename = "chat_interactions.xlsx"

    if not os.path.exists(filename):
        # Create new Excel file with proper columns
        df = pd.DataFrame(columns=[
            'Timestamp',
            'Session_ID',
            'User_Query',
            'AI_Response',
            'Response_Time_Seconds',
            'Query_Length',
            'Response_Length'
        ])
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"✅ Created new Excel database: {filename}")
    else:
        print(f"✅ Excel database exists: {filename}")

    return filename

In [ ]:
def save_interaction_to_excel(user_query, ai_response, response_time, session_id=None):
    '''
    Save chat interaction data to Excel file with timestamp, query/response metrics, and session tracking.
    Appends new interaction row to existing Excel database, handles file creation/reading errors, tracks conversation analytics.
    '''
    try:
        filename = "chat_interactions.xlsx"

        # Create new row data
        new_row = {
            'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Session_ID': session_id or f"session_{int(time.time())}",
            'User_Query': user_query,
            'AI_Response': ai_response,
            'Response_Time_Seconds': round(response_time, 2),
            'Query_Length': len(user_query),
            'Response_Length': len(ai_response)
        }

        # Read existing data or create new DataFrame
        if os.path.exists(filename):
            try:
                df = pd.read_excel(filename, engine='openpyxl')
                # Append new row using loc to avoid FutureWarning
                df.loc[len(df)] = new_row
            except Exception as read_error:
                print(f"Warning: Could not read existing file ({read_error}), creating new one")
                df = pd.DataFrame([new_row])
        else:
            # Create new DataFrame with first row
            df = pd.DataFrame([new_row])

        # Save to Excel
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"💾 Saved interaction to {filename} (Total rows: {len(df)})")

    except Exception as e:
        print(f"❌ Error saving to Excel: {e}")
        # Don't let Excel errors break the chat
        pass

##Llama Interactions

In [ ]:
def get_llm_response(prompt, max_retries=3):
    '''
    Get response from private Llama-3-8B model with retry logic and error handling.
    Attempts multiple requests with 2-second delays between retries, returns graceful error message if all attempts fail.
    '''
    for attempt in range(max_retries):
        try:
            messages = [{"role": "user", "content": prompt}]
            response = client.chat.completions.create(
                model="llama-3-8b-instruct",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retry
                print(f"Attempt {attempt + 1} failed: {e}. Retrying...")
            else:
                print(f"All attempts failed. Error: {e}")
                return f"Sorry, I'm having trouble connecting to the AI service. Error: {str(e)}"

In [ ]:
def chat_with_llm(user_input, chat_history, session_state):
    '''
    Handle complete chat interaction flow with LLM including session management, response timing, and Excel logging.
    Manages chat history, generates session IDs, calls LLM with error handling, saves all interactions and errors to Excel database.
    '''
    if not user_input.strip():
        return chat_history, "", session_state

    # Generate session ID if not exists
    if not session_state.get("session_id"):
        session_state["session_id"] = f"session_{int(time.time())}"

    # Add user message to history
    chat_history.append({"role": "user", "content": user_input})

    # Get LLM response with timing
    start_time = time.time()
    try:
        ai_response = get_llm_response(user_input)
        response_time = time.time() - start_time

        # Add AI response to history
        chat_history.append({"role": "assistant", "content": ai_response})

        # Save to Excel
        save_interaction_to_excel(
            user_query=user_input,
            ai_response=ai_response,
            response_time=response_time,
            session_id=session_state["session_id"]
        )

    except Exception as e:
        response_time = time.time() - start_time
        error_message = f"Error: {str(e)}"
        chat_history.append({"role": "assistant", "content": error_message})

        # Save error to Excel too
        save_interaction_to_excel(
            user_query=user_input,
            ai_response=error_message,
            response_time=response_time,
            session_id=session_state["session_id"]
        )

    return chat_history, "", session_state

In [ ]:
def clear_chat(session_state):
    """Clear the chat history but keep session info"""
    return [], "", session_state

def test_connection():
    """Test the LLM connection"""
    try:
        test_response = get_llm_response("Hello, can you confirm you're working?")
        return f"✅ Connection successful! Response: {test_response[:100]}..."
    except Exception as e:
        return f"❌ Connection failed: {str(e)}"

In [ ]:
def view_excel_stats():
    """
    View statistics from the Excel database
    """
    try:
        filename = "chat_interactions.xlsx"
        if not os.path.exists(filename):
            return "📊 No chat data found. Start chatting to see statistics!"

        df = pd.read_excel(filename, engine='openpyxl')

        if df.empty:
            return "📊 No interactions recorded yet."

        total_interactions = len(df)
        unique_sessions = df['Session_ID'].nunique() if 'Session_ID' in df.columns else 0
        avg_response_time = df['Response_Time_Seconds'].mean() if 'Response_Time_Seconds' in df.columns else 0
        avg_query_length = df['Query_Length'].mean() if 'Query_Length' in df.columns else 0

        # Recent interactions
        recent = df.tail(5)[['Timestamp', 'User_Query', 'Response_Time_Seconds']].copy()
        recent['User_Query'] = recent['User_Query'].apply(
            lambda x: str(x)[:50] + '...' if len(str(x)) > 50 else str(x)
        )

        stats = f"""📊 **Chat Database Statistics**

**Overall Stats:**
- Total Interactions: {total_interactions}
- Unique Sessions: {unique_sessions}
- Average Response Time: {avg_response_time:.2f} seconds
- Average Query Length: {avg_query_length:.0f} characters

**Recent Interactions:**
{recent.to_string(index=False)}

**File Location:** {os.path.abspath(filename)}
        """

        return stats

    except Exception as e:
        return f"❌ Error reading Excel database: {e}"

#UI MAIN

In [ ]:
def create_basic_interface():
  '''
  Create simple Gradio chat interface with Excel logging for basic LLM interactions.
  Initializes Excel database, provides chat UI with send/clear functions, connection testing, stats viewing, and example prompts.
  '''
    # Initialize Excel database
    initialize_excel_database()

    with gr.Blocks(title="Basic LLM Chat Interface") as demo:

        gr.Markdown("# 🤖 Basic LLM Chat Interface")
        gr.Markdown("Simple chat interface using your private LLM server with Excel logging")

        # Session state to track user sessions
        session_state = gr.State({"session_id": None})

        # Chat interface
        chatbot = gr.Chatbot(
            label="Chat with AI",
            type="messages",
            height=400,
            show_label=True
        )

        # Input area
        with gr.Row():
            with gr.Column(scale=4):
                msg = gr.Textbox(
                    label="Your message",
                    placeholder="Type your message here...",
                    lines=2,
                    max_lines=5
                )
            with gr.Column(scale=1):
                send_btn = gr.Button("🚀 Send", variant="primary", size="lg")
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")

        # Management area
        with gr.Row():
            test_btn = gr.Button("🔧 Test Connection")
            stats_btn = gr.Button("📊 View Excel Stats")

        with gr.Row():
            connection_status = gr.Textbox(
                label="Connection Status",
                interactive=False,
                lines=2
            )
            excel_stats = gr.Textbox(
                label="Excel Database Stats",
                interactive=False,
                lines=10,
                visible=False
            )

        # Event handlers
        def handle_submit(user_input, history, session):
            return chat_with_llm(user_input, history, session)

        def handle_clear(session):
            return clear_chat(session)

        # Send button click
        send_btn.click(
            handle_submit,
            inputs=[msg, chatbot, session_state],
            outputs=[chatbot, msg, session_state]
        )

        # Enter key press
        msg.submit(
            handle_submit,
            inputs=[msg, chatbot, session_state],
            outputs=[chatbot, msg, session_state]
        )

        # Clear button
        clear_btn.click(
            handle_clear,
            inputs=[session_state],
            outputs=[chatbot, msg, session_state]
        )

        # Test connection
        test_btn.click(
            test_connection,
            outputs=[connection_status]
        )

        # View Excel stats
        def show_stats():
            stats = view_excel_stats()
            return gr.update(visible=True), stats

        stats_btn.click(
            show_stats,
            outputs=[excel_stats, excel_stats]
        )

        # Examples
        gr.Examples(
            examples=[
                ["Hello! How are you today?"],
                ["Can you explain quantum computing in simple terms?"],
                ["Write a short poem about programming"],
                ["What's the capital of France?"],
                ["Help me understand machine learning"]
            ],
            inputs=[msg]
        )

    return demo

In [ ]:
def test_llm():
    """Test function to verify LLM is working"""
    print("Testing LLM connection...")
    response = get_llm_response("Hello! Please respond with a brief greeting.")
    print(f"LLM Response: {response}")
    return response

if __name__ == "__main__":
    # Test the connection first
    print("🔧 Testing LLM connection...")
    test_result = test_llm()

    if "Error:" not in test_result:
        print("✅ LLM connection successful!")
        print("📊 Initializing Excel database...")
        initialize_excel_database()
        print("🚀 Starting Gradio interface...")

        # Create and launch the interface
        demo = create_basic_interface()
        demo.launch(
            share=True)
    else:
        print("❌ LLM connection failed!")
        print(f"Error: {test_result}")

🔧 Testing LLM connection...
Testing LLM connection...
LLM Response: Hello! It's nice to meet you!
✅ LLM connection successful!
📊 Initializing Excel database...
✅ Created new Excel database: chat_interactions.xlsx
🚀 Starting Gradio interface...
✅ Excel database exists: chat_interactions.xlsx
* Running on local URL:  http://127.0.0.1:7874
* Running on public URL: https://670ebae4b9d06494aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


💾 Saved interaction to chat_interactions.xlsx (Total rows: 1)
💾 Saved interaction to chat_interactions.xlsx (Total rows: 2)
💾 Saved interaction to chat_interactions.xlsx (Total rows: 3)
💾 Saved interaction to chat_interactions.xlsx (Total rows: 4)
💾 Saved interaction to chat_interactions.xlsx (Total rows: 5)
💾 Saved interaction to chat_interactions.xlsx (Total rows: 6)
